# Parameters

In [1]:
data_size = 2000; dataset_views_to_consider = 'all'; seed = 42
d_input_enc=2000; nb_classes_dec=33; class_weights=[]; d_model_enc_dec=1024; d_ff_enc_dec=1024; n_heads_enc_dec=16; n_layers_enc=2; n_layers_dec=2; activation="relu"; dropout=0.1 
model_params = {
    "d_input_enc": 2000, 
    "lr": 6.033193735866575e-05,
    "nb_classes_dec": 33,
    "early_stopping": True,
    "dropout": 0.16171970479206027,
    "weight_decay": 5.4598394312421854e-05,
    "activation": "relu",
    "optimizer": "Adam",
    "lr_scheduler": "cosine_with_restarts",
    "loss": "ce",
    "n_epochs": 1, 
    "batch_size": 256,
    "class_weights":[4.03557312, 0.85154295, 0.30184775, 1.18997669, 8.25050505,
            0.72372851, 7.73484848, 1.81996435, 0.62294082, 0.61468995,
            4.07992008, 0.49969411, 1.07615283, 1.85636364, 0.7018388 ,
            0.84765463, 0.60271547, 0.62398778, 4.26750261, 0.61878788,
            1.89424861, 1.98541565, 0.65595888, 2.05123054, 1.37001006,
            0.77509964, 0.76393565, 2.67102681, 0.64012539, 2.94660895,
            0.64012539, 6.51355662, 4.64090909],
    "d_model_enc_dec": 512,
    "n_heads_enc_dec": 16,
    "n_layers_enc": 10,
    "n_layers_dec": 1
}
d_ff_enc_dec_value = model_params["d_model_enc_dec"] * 4
model_params["d_ff_enc_dec"] = d_ff_enc_dec_value

fit_params = {
    "nb_ckpts":1, 
    "verbose":1
}

predict_params = {
    "nb_ckpts":1, 
    "scores_fname": "transformer_scores.json"
}

training_params = {
    "model_params": model_params,
    "fit_params": fit_params,
    "predict_params": predict_params,
    "data_size": int('2000'),
    "dataset_views_to_consider": 'all',
#     "exp_type": "data_aug",
    "exp_type": "data_aug",
    "seed": 42
}
output_path = './'
# salloc --time=02:00:00 --nodes=1 --gres=gpu:1 --ntasks-per-node=32  --mem-per-cpu=127000M  --account=rrg-corbeilj-ac
# salloc --time=06:00:00 --gres=gpu:v100:1 --cpus-per-task=3 --mem=32000M --account=rrg-corbeilj-ac

# Import section

In [2]:
import argparse
import os
import json
import torch
import random
import natsort
import numpy as np
import pandas as pd
from tqdm import tqdm
from argparse import Namespace


import optuna
from optuna.study import StudyDirection
from packaging import version
from multiomic_modeling.models.trainer import MultiomicTrainer, MultiomicTrainerMultiModal
from multiomic_modeling.models.base import PatientPruner
from multiomic_modeling.models.base import BaseTrainer
from multiomic_modeling.data.data_loader import MultiomicDatasetDataAug, MultiomicDatasetNormal, MultiomicDatasetBuilder, SubsetRandomSampler
from multiomic_modeling.models.models import MultiomicPredictionModel, MultiomicPredictionModelMultiModal
from multiomic_modeling.models.utils import expt_params_formatter, c_collate
from multiomic_modeling.loss_and_metrics import ClfMetrics, NumpyEncoder
from multiomic_modeling.utilities import params_to_hash
from multiomic_modeling.torch_utils import to_numpy, totensor, get_optimizer
from multiomic_modeling import logging
from torch.utils.data import DataLoader
from transformers.optimization import Adafactor, AdamW, \
    get_cosine_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

import pytorch_lightning as pl
import torch

if version.parse(pl.__version__) < version.parse("1.0.2"):
    raise RuntimeError("PyTorch Lightning>=1.0.2 is required for this example.")



In [3]:
# dataset = MultiomicDatasetNormal(data_size=data_size, views_to_consider=dataset_views_to_consider)

# train, test, valid = MultiomicDatasetBuilder().multiomic_data_normal_builder(dataset=dataset, 
#                                                                                          test_size=0.2, 
#                                                                                          valid_size=0.1, 
#                                                                                          random_state=seed)

# dataset_augmented = MultiomicDatasetDataAug(train_dataset=train, data_size=data_size, views_to_consider=dataset_views_to_consider)

# train_augmented = MultiomicDatasetBuilder.multiomic_data_aug_builder(augmented_dataset=dataset_augmented)

In [4]:
# print(len(train.indices))
# print(train[0])
# print(len(train[0]))
# print(len(train[0][0]))

# print(len(train_augmented.indices))
# print(train_augmented[0])
# print(len(train_augmented[0]))
# print(len(train_augmented[0][0]))

In [5]:
model = MultiomicTrainerMultiModal.run_experiment(**training_params, output_path=output_path)

>>> Training configuration : 
{
  "data_size": 2000,
  "dataset_views_to_consider": "all",
  "exp_type": "data_aug",
  "fit_params": {
    "nb_ckpts": 1,
    "verbose": 1
  },
  "kwargs": {},
  "model_params": {
    "activation": "relu",
    "batch_size": 256,
    "class_weights": [
      4.03557312,
      0.85154295,
      0.30184775,
      1.18997669,
      8.25050505,
      0.72372851,
      7.73484848,
      1.81996435,
      0.62294082,
      0.61468995,
      4.07992008,
      0.49969411,
      1.07615283,
      1.85636364,
      0.7018388,
      0.84765463,
      0.60271547,
      0.62398778,
      4.26750261,
      0.61878788,
      1.89424861,
      1.98541565,
      0.65595888,
      2.05123054,
      1.37001006,
      0.77509964,
      0.76393565,
      2.67102681,
      0.64012539,
      2.94660895,
      0.64012539,
      6.51355662,
      4.64090909
    ],
    "d_ff_enc_dec": 2048,
    "d_input_enc": 2000,
    "d_model_enc_dec": 512,
    "dropout": 0.16171970479206027,
  

/home/maoss2/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Checkpoint directory ./649c350f57b9ffb4277a24511a90696abae3269a/checkpoints exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name    | Type                               | Params
---------------------------------------------------------------
0 | network | MultiomicPredictionModelMultiModal | 42.0 M
---------------------------------------------------------------
42.0 M    Trainable params
0         Non-trainable params
42.0 M    Total params
168.010   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

6it [00:00, 29.20it/s]

<class 'list'>
2
[tensor([[[ 3.4400e-02,  3.4400e-02,  3.4400e-02,  ..., -8.3000e-03,
          -2.3100e-02, -2.3100e-02],
         [ 4.6162e-01,  4.4909e-01,  9.5354e-01,  ...,  7.6624e-02,
           3.6316e-01,  5.9298e-01],
         [ 2.8100e+00,  5.1900e+00,  6.6900e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.2170e+01,  3.3200e+00,  0.0000e+00,  ...,  6.5000e+00,
           9.7800e+00,  4.8000e+00],
         [ 4.3544e-01,  2.9133e+00, -1.5067e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-9.3380e-01, -9.3380e-01, -9.3380e-01,  ...,  1.2920e-01,
           1.1840e-01,  1.1840e-01],
         [ 4.0030e-02,  9.6437e-01,  9.1406e-01,  ...,  6.9855e-01,
           7.4652e-01,  5.2938e-01],
         [ 1.2160e+01,  5.6000e+00,  6.2700e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.1700e+01,  6.9200e+00,  1.2120e+01,  ...,  8.9600e+00,
           1.0260e+01,  6.6900e+00],
         [ 2.9714e+00,  2.2501e+00,

10it [00:00, 31.27it/s]
0it [00:00, ?it/s]

tensor([22,  2, 16, 26, 10, 25,  8, 25, 16, 28,  1,  2,  5, 30, 28,  2,  2, 29,
        16,  0,  9, 27, 26, 25, 11, 22, 19, 30, 28,  2,  1, 12, 23, 19,  3,  2,
         5, 16, 28,  2,  2, 14, 12, 19, 30, 22, 11, 17,  2,  7,  8, 29, 17,  3,
         2, 14, 26,  3,  3, 15, 23,  9, 17, 28, 26,  8, 11,  5, 19, 25, 12, 16,
        17,  5,  8, 29, 25, 18, 15, 26, 28, 17, 30, 11, 12,  2, 25,  2, 11, 31,
        23, 28,  8,  6, 26, 22, 20, 11, 30, 15, 17, 30, 14, 12, 17,  9,  2, 17,
        15, 13, 30,  7,  5, 26, 17, 21,  2, 22,  2, 15,  3, 17,  2,  2, 17, 24,
         9,  9,  2, 19, 14, 17, 26,  2, 28, 28, 16,  3, 25, 11, 22, 28, 10, 11,
         8, 24, 11, 19, 18, 25,  9, 30,  8,  2,  2, 12,  3, 11, 26, 21, 25, 16,
        30, 11,  0, 17, 22,  9,  8, 10,  1,  2,  2, 15,  1, 26,  3, 12, 17, 12,
        10, 12, 12, 14, 26,  8, 24,  2, 11,  2, 15, 28,  8, 12,  2, 10,  5, 19,
         6,  8,  8, 30, 30,  8,  2, 30, 17, 22,  8, 17, 30, 12,  0, 28,  1, 15,
        30, 25, 15, 28, 15,  1,  3, 13, 

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not tuple